# Keras demo

In [1]:
import numpy as np
import cv2
import sys
import os
import glob
import pickle
import time

In [2]:
# Parameters
gpu_enabled = False
model_path = '../final_saved_models'

In [3]:
# COCO Class names
class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']

In [4]:
def keras_model_test(model_type, model_path, image_num, gpu_enabled = False):


    if not gpu_enabled:
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        platform = 'cpu'
    else:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        platform = 'gpu'
    
    classes=len(class_names)    
    
    if model_type=='vgg16':
        model_final = 'vgg16_coco_120.p'
        from graphs import vgg_graph
        model, base_size = vgg_graph(classes, architecture = "vgg16")
    elif model_type=='resnet50':
        model_final = 'resnet50_coco_120.p'   
        from graphs import resnet_graph
        model, base_size = resnet_graph(classes, architecture = "resnet50")
            
    model_path = os.path.join(model_path,model_final)
    wlayer = pickle.load(open(model_path,"rb"))
    model.set_weights(wlayer)
        
    #warm-up
    y_per = model.predict(np.zeros((1,base_size,base_size,3) ,np.float32))
    
    image_name = "./test_data/%d/i_%d_%02d.png"%(base_size,base_size, image_num)    
    a = cv2.imread(image_name)
    
    x_test = np.reshape(np.float32(a)/255.0-0.5, (1, base_size,base_size, 3))
    
    start_time = time.time()
    y_per = model.predict(x_test)
    ellaped = time.time()-start_time
    
    print("class: %s, prob: %2.2f ,elapsed time: %2.2f sec"%(class_names[int(np.argmax(y_per))], np.max(y_per), ellaped))
    

# VGG16 Model

In [5]:
keras_model_test('vgg16', model_path, image_num = 0, gpu_enabled=gpu_enabled)

Using TensorFlow backend.


class: bowl, prob: 1.00 ,elapsed time: 0.27 sec


# Resnet50 model

In [6]:
keras_model_test('resnet50', model_path, image_num = 0, gpu_enabled=gpu_enabled)

class: bowl, prob: 1.00 ,elapsed time: 0.15 sec
